# Домашняя работа №1
1. Посмотреть документацию к API GitHub, разобраться как вывести список репозиториев для конкретного пользователя, сохранить JSON-вывод в файле *.json.

In [1]:
import requests
import json
from pprint import pprint
import numpy as np
import pandas as pd

In [2]:
params = {
    'Authorization': 'Bearer' # с ключом пробовал, но здесь убрал
}

url = 'https://api.github.com/users/SergeyPresnyakov/repos'

response = requests.get(url=url, params=params)

with open('repo_presnyakov.json', 'w') as file:
    json.dump([item['full_name'] for item in response.json()], file)


2. Работа будет состоять с недокументированным API. Нужно ввести релевантный запрос на сайте https://www.delivery-club.ru/search  
(а_1) из предложенных точек с помощью API найти долю (в %) с бесплатной и платной доставкой. Для каждой категории рассчитать среднюю и минимальную стоимость заказа.  

In [3]:
url = 'https://api.delivery-club.ru/api1.2/vendors/search?latitude=51.771114&longitude=55.185412&query=%D0%BF%D0%B5%D0%BB%D1%8C%D0%BC%D0%B5%D0%BD%D0%B8&cacheBreaker=1664137218'

response = requests.get(url=url).json()
response

{'found': 16,
 'search_id': '9a77d747-0b52-43a4-b140-cc148213ee9d',
 'total': 16,
 'vendors': [{'alias': 'prospjekt_Pobjedy155',
   'allowPreorder': True,
   'categoryId': 5,
   'chain': {'alias': 'metro_ornbg', 'id': {'primary': '46840'}},
   'cover': {'1200': 'https://img2.delivery-club.ru/discovery/678e157bb8c959160420ecf10fced7a3_3161936033571404640.jpg',
    '1600': 'https://img2.delivery-club.ru/discovery/6f83b8a8027bf1ba3ee780adcc0088be_1544215265531150625.jpg',
    '2048': 'https://img2.delivery-club.ru/discovery/0b3c86eff53119506b073a947b344d76_7485277845224726427.jpg',
    '480': 'https://img2.delivery-club.ru/discovery/1fc18c4ee16ea8703da7ae4e8f35536c_1458678175561432897.jpg',
    '720': 'https://img2.delivery-club.ru/discovery/8ae0cf1c46dd6ca551db58bb1926e6bc_4428742896577266346.jpg'},
   'cuisines': [],
   'delivery': {'available': True,
    'minOrderPrice': {'value': 1000},
    'price': {'value': 0},
    'services': ['delivery'],
    'time': 'сегодня'},
   'id': {'primary

In [4]:
# Общее количество вендоров
num_vendor_comm = len(response['vendors'])
num_vendor_comm 

16

In [5]:
# Количесвто вендоров с платной доставкой
num_vendor_paid = len([item['delivery']['price']['value'] for item in response['vendors'] if item['delivery']['price']['value'] > 0])
num_vendor_paid

8

In [6]:
print(f'Доля вендоров с платной доставкой составляет {num_vendor_paid / num_vendor_comm * 100}%')

Доля вендоров с платной доставкой составляет 50.0%


(a_2) Для каждой категории рассчитать среднюю и минимальную стоимость заказа.  

Есть две категории заведений по стоимости доставки:  
- с платной доставкой  
- с бесплатной доставкой  

Есть две категории заведений по виду деятельности:  

- Рестораном считаю заведения с ID = 1
- Магазином считаю заведения с ID >1


Стоимость заказа рассчитывается как стоимость заказа для каждого заведения с бесплатной доставкой и как сумма доставки и минимальная стоимость блюда для заведений с платной доставкой.  
Рассчитаем среднюю и минимальную стоимость заказа отдельно для заведений с платной доставкой и для заведений с бесплатной доставкой

In [7]:
# Cредняя стоимость заказа для заведений с бесплатной доставкой
np.mean([item['delivery']['minOrderPrice']['value'] for item in response['vendors'] if item['delivery']['price']['value'] == 0])

837.5

In [8]:
# Минимальная стоимость заказа для заведений с бесплатной доставкой
np.min([item['delivery']['minOrderPrice']['value'] for item in response['vendors'] if item['delivery']['price']['value'] == 0])

500

In [9]:
# Стоимость доставки + стоимость заказа в случае платной доставки (1 рубль) + минимальная стоимость блюда для заведений с платной доставкой
data = {item['name']: [item['delivery']['price']['value'] + item['delivery']['minOrderPrice']['value'], np.min([description['price']['value'] for description in item['products']])] 
       for item in response['vendors'] if ((item['delivery']['price']['value'] > 0) & (np.sum([description['price']['value'] for description in item['products']]) > 0))}
data

{'Феникс': [130, 229],
 'Хорошая компания': [160, 324],
 'Хорошая компания Экспресс': [200, 199],
 'Ели Пельмени': [130, 200],
 'Антракт': [140, 170],
 'Инфинити': [200, 380],
 'Пекарня-пироговая Пироги': [200, 136]}

In [10]:
# Тоже самое, но уже просуммировано
data_2 = {item['name']: item['delivery']['price']['value'] + item['delivery']['minOrderPrice']['value'] + np.min([description['price']['value'] for description in item['products']]) 
       for item in response['vendors'] if ((item['delivery']['price']['value'] > 0) & (np.sum([description['price']['value'] for description in item['products']]) > 0))}
data_2

{'Феникс': 359,
 'Хорошая компания': 484,
 'Хорошая компания Экспресс': 399,
 'Ели Пельмени': 330,
 'Антракт': 310,
 'Инфинити': 580,
 'Пекарня-пироговая Пироги': 336}

In [11]:
# Cредняя стоимость заказа для заведений с платной доставкой
np.mean([data_2[key] for key in data_2])

399.7142857142857

In [12]:
#  Минимальная стоимость заказа для заведений с платной доставкой
np.min([data_2[key] for key in data_2])

310

(б) для каждой из категорий из пункта (а) рассчитать долю (в %) магазинов и ресторанов

In [13]:
# Определим вид деятельности и стоимость доставки для каждого заведения
data_3 = {item['name']: [item['categoryId'], item['delivery']['price']['value']] for item in response['vendors']}
data_3

{'METRO': [5, 0],
 'Феникс': [1, 129],
 'Хорошая компания': [1, 159],
 'Хорошая компания Экспресс': [1, 199],
 'Шампуроff': [1, 0],
 'Бегемот': [1, 429],
 'КОСМОС': [1, 0],
 'Рикко. Твоя пицца.': [1, 0],
 'Ели Пельмени': [1, 129],
 'Ready Meal': [1, 0],
 'Орендвор': [1, 0],
 'Жемчужина': [1, 0],
 'Антракт': [1, 139],
 'Инфинити': [1, 199],
 'Добрый Кулинар': [1, 0],
 'Пекарня-пироговая Пироги': [1, 199]}

In [14]:
# Доля ресторанов (в %) среди заведений с бесплатной доставкой
len([key for key, value in data_3.items() if ((value[0] == 1) & (value[1] == 0))]) / len([key for key, value in data_3.items() if value[1] == 0]) * 100  

87.5

In [15]:
# Доля магазинов (в %) среди заведений с бесплатной доставкой
100 - len([key for key, value in data_3.items() if ((value[0] == 1) & (value[1] == 0))]) / len([key for key, value in data_3.items() if value[1] == 0]) * 100  

12.5

In [16]:
# Доля магазинов (в %) среди заведений с платной доставкой
len([key for key, value in data_3.items() if ((value[0] == 5) & (value[1] > 0))]) / len([key for key, value in data_3.items() if value[1] > 0]) * 100  

0.0

In [17]:
# Доля рестораов  (в %) среди заведений с платной доставкой
len([key for key, value in data_3.items() if ((value[0] == 1) & (value[1] > 0))]) / len([key for key, value in data_3.items() if value[1] > 0]) * 100  

100.0